In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import datetime
import copy
import time

import sys
sys.path.append('../../')

from data.processing import get_data

import models

from main.seir.main import single_fitting_cycle
from main.seir.forecast import create_all_trials_csv, create_decile_csv_new
from utils.generic.create_report import save_dict_and_create_report
from utils.generic.config import read_config, make_date_key_str
from utils.generic.enums import Columns
from utils.fitting.loss import Loss_Calculator
from utils.generic.logger import log_wandb, log_mlflow
from viz import plot_forecast, plot_top_k_trials, plot_ptiles
from viz.uncertainty import plot_beta_loss

import yaml
import wandb

In [ ]:
predictions_dict = {}

In [ ]:
config_filename = 'mcmc_simulate.yaml'
config = read_config(config_filename)

wandb_config = read_config(config_filename, preprocess=False)
wandb_config = make_date_key_str(wandb_config)

In [ ]:
timestamp = datetime.datetime.now()
output_folder = '../../misc/reports/{}'.format(timestamp.strftime("%Y_%m%d_%H%M%S"))

## Perform Fit

In [ ]:
predictions_dict = single_fitting_cycle(**copy.deepcopy(config['fitting']))

## Loss Dataframe

In [ ]:
from viz.fit import plot_histogram
fig,axs = plt.subplots(4,2,figsize = (10,20))
plot_histogram(predictions_dict,fig = fig,axs=axs)

In [ ]:
predictions_dict['df_loss']

## Plot Best Forecast

In [ ]:
predictions_dict['forecasts'] = {}
predictions_dict['forecasts']['best'] = predictions_dict['trials']['predictions'][0]

predictions_dict['plots']['forecast_best'] = plot_forecast(predictions_dict, 
                                                            which_compartments=config['fitting']['loss']['loss_compartments'],
                                                            error_bars=False, **config['plotting'])

## Process trials + Find best beta

In [ ]:
uncertainty_args = {'predictions_dict': predictions_dict,  "variable_param_ranges" :config['fitting']['variable_param_ranges'],
                     **config['uncertainty']['uncertainty_params']}
uncertainty = config['uncertainty']['method'](**uncertainty_args)

In [ ]:
predictions_dict['plots']['beta_loss'], _ = plot_beta_loss(uncertainty.dict_of_trials)

In [ ]:
uncertainty_forecasts = uncertainty.get_forecasts()
for key in uncertainty_forecasts.keys():
    predictions_dict['forecasts'][key] = uncertainty_forecasts[key]['df_prediction']
    
predicions_dict['forecasts']['ensemble_mean'] = uncertainty.ensemble_mean_forecast

## Plot Top k Trials

In [ ]:
kforecasts = plot_top_k_trials(predictions_dict, k=config['plotting']['num_trials_to_plot'],
                               which_compartments=config['plotting']['plot_topk_trials_for_columns'])
                               
predictions_dict['plots']['forecasts_topk'] = {}
for column in config['plotting']['plot_topk_trials_for_columns']:
    predictions_dict['plots']['forecasts_topk'][column.name] = kforecasts[column]

In [ ]:
predictions_dict['beta'] = uncertainty.beta
predictions_dict['beta_loss'] = uncertainty.beta_loss
predictions_dict['deciles'] = uncertainty_forecasts

## Plot Deciles Forecasts

In [ ]:
for fits_to_plot in config['plotting']['pair_fits_to_plot']:
    predictions_dict['plots'][f'forecast_{fits_to_plot[0]}_{fits_to_plot[1]}'] = plot_forecast(
        predictions_dict, which_compartments=config['fitting']['loss']['loss_compartments'],
        fits_to_plot=fits_to_plot, **config['plotting'])

In [ ]:
ptiles_plots = plot_ptiles(predictions_dict, which_compartments=config['plotting']['plot_ptiles_for_columns'])
predictions_dict['plots']['forecasts_ptiles'] = {}
for column in config['plotting']['plot_ptiles_for_columns']:
    predictions_dict['plots']['forecasts_ptiles'][column.name] = ptiles_plots[column]

## Create Report

In [ ]:
save_dict_and_create_report(predictions_dict, config, ROOT_DIR=output_folder, config_filename=config_filename)

## Create Output CSV

In [ ]:
df_output = create_decile_csv_new(predictions_dict)
df_output.to_csv(f'{output_folder}/deciles.csv')

## Create All Trials Output

In [ ]:
df_all = create_all_trials_csv(predictions_dict)
df_all.to_csv(f'{output_folder}/all_trials.csv')

## Log on W&B

In [ ]:
wandb.init(project="covid-modelling", config=wandb_config)

In [ ]:
log_wandb(predictions_dict)

## Log on MLFlow

In [ ]:
log_mlflow(config['logging']['experiment_name'], run_name=config['logging']['run_name'], artifact_dir=output_folder)